In [1]:
import pandas as pd
from datetime import datetime
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
power_measurements_Header = [];

power_measurements = []
power_stats = []
fork_exit_exec_data = []

current = 0;

#Replace this with the dump file's path
with open('/content/drive/MyDrive/HSBC/march-to-sustainability/output.txt') as file:
    
    for line in file:

      line = line.split()

      if (len(line) > 0 and line[0]=='Time'):
        power_measurements_Header = line
        current = 1
        continue

      elif (len(line) > 0 and line[0]=='--------'):
        current = 2
        continue

      elif (len(line) > 0 and line[0]=='Log'):
        current = 3
        continue

      if (current == 1):
        power_measurements.append(line)

      elif (current == 2):
        if (line[0] == "Summary:"):
          current = 0
          continue
        power_stats.append(line)
      
      elif (current == 3):
        fork_exit_exec_data.append(line)



In [4]:
df_power_measurements = pd.DataFrame(power_measurements, columns = power_measurements_Header)
df_power_measurements

,Time,User,Nice,Sys,Idle,IO,Run,Ctxt/s,IRQ/s,Fork,Exec,Exit,Watts
0,20:20:33,0.4,0.7,0.1,98.6,0.2,1,744,343,3,0,3,8.35
1,20:20:43,0.8,0.0,0.1,98.8,0.2,1,796,353,2,0,2,8.26
2,20:20:53,0.9,0.0,0.4,98.6,0.1,1,1503,4416,0,0,2,8.31
3,20:21:03,0.3,0.0,0.2,99.4,0.1,1,763,337,1,0,1,8.20
4,20:21:13,0.3,0.0,0.2,99.4,0.1,1,734,325,0,0,0,8.11
5,20:21:23,0.4,0.0,0.1,99.3,0.2,1,756,335,1,0,2,8.02
6,20:21:33,0.3,0.0,0.1,99.5,0.1,1,722,322,2,0,2,7.98
7,20:21:43,0.2,0.0,0.2,99.5,0.1,1,721,321,0,0,3,7.90
8,20:21:53,0.5,0.0,0.1,99.2,0.1,1,743,334,2,0,0,7.85
9,20:22:03,0.3,0.0,0.1,99.5,0.1,1,730,326,0,0,2,7.82


Change the path to whatever you want it to be in your google drive - this will output the basic measurements shown above

In [5]:
basic_power_measurements_path = '/content/drive/MyDrive/HSBC/march-to-sustainability/basic_power_measurements.csv'

with open(basic_power_measurements_path, 'w', encoding = 'utf-8-sig') as f:
  df_power_measurements.to_csv(f)

#DON'T RERUN THIS BLOCK AFTER FIRST TIME

In [6]:
timeStamps = df_power_measurements["Time"].tolist()
wattages = df_power_measurements["Watts"].tolist()

current_timestamp_index = 0;
timeStamp_to_process_dict = {}

timeStamp_to_process_dict[timeStamps[current_timestamp_index]] = []

for item in fork_exit_exec_data:

  #check if current item pertains to an unknown process, if yes we can skip the iteration
  if (item[len(item)-1] == '(<unknown>)'):
    continue

  #timestamp for item
  item_time = datetime.strptime(item[0], '%H:%M:%S')
  #timestamp from main table
  key_time_stamp = datetime.strptime(timeStamps[current_timestamp_index], '%H:%M:%S')

  if item_time <= key_time_stamp:
    #insert into corresponding dictionary entry
    timeStamp_to_process_dict[timeStamps[current_timestamp_index]].append(item)
  
  else:
    #assign random wattages 
    num_timestamps = len(timeStamp_to_process_dict[timeStamps[current_timestamp_index]])
    weightageList = random.sample(range(1, 100), num_timestamps)
    weightSum = sum(weightageList)
    total_watts = wattages[current_timestamp_index]

    for i in range(num_timestamps):
      watts = float(total_watts)*weightageList[i]/weightSum
      #changed - toInsert = str(watts) + " Watts"
      toInsert = watts
      timeStamp_to_process_dict[timeStamps[current_timestamp_index]][i].insert(1, toInsert)

    #increment timestamp index and assign empty array to value
    current_timestamp_index += 1
    timeStamp_to_process_dict[timeStamps[current_timestamp_index]] = []

    if item_time <= key_time_stamp:
      #insert into corresponding dictionary entry
      timeStamp_to_process_dict[timeStamps[current_timestamp_index]].append(item)

#assign random wattages 
num_timestamps = len(timeStamp_to_process_dict[timeStamps[current_timestamp_index]])
weightageList = random.sample(range(1, 100), num_timestamps)
weightSum = sum(weightageList)
total_watts = wattages[current_timestamp_index]

for i in range(num_timestamps):
  watts = float(total_watts)*weightageList[i]/weightSum
  toInsert = watts #changed
  timeStamp_to_process_dict[timeStamps[current_timestamp_index]][i].insert(1, toInsert)


In [7]:
timeStamp_to_process_dict

{'20:20:33': [['20:20:26',
   2.7427007299270074,
   'fork:',
   'parent',
   'pid=3218',
   '->',
   'clone',
   'pid=5037',
   '(/usr/lib/tracker/tracker-miner-fs)'],
  ['20:20:30',
   1.8284671532846715,
   'fork:',
   'parent',
   'pid=2886',
   '->',
   'clone',
   'pid=5038',
   '(gpg-agent)'],
  ['20:20:30',
   1.9198905109489048,
   'fork:',
   'parent',
   'pid=2886',
   '->',
   'clone',
   'pid=5039',
   '(gpg-agent)'],
  ['20:20:30',
   1.676094890510949,
   'exit:',
   'pid=5038',
   'exit_code=0',
   '(gpg-agent)'],
  ['20:20:30',
   0.18284671532846714,
   'exit:',
   'pid=5039',
   'exit_code=0',
   '(gpg-agent)']],
 '20:20:43': [['20:20:42',
   2.8320000000000003,
   'fork:',
   'parent',
   'pid=4538',
   '->',
   'clone',
   'pid=5041',
   '(java)'],
  ['20:20:42',
   5.428,
   'exit:',
   'pid=5037',
   'exit_code=0',
   '(/usr/lib/tracker/tracker-miner-fs)']],
 '20:20:53': [['20:20:50',
   8.31,
   'exit:',
   'pid=5040',
   'exit_code=0',
   '(/usr/bin/whoopsie)']

In [29]:
timeStamp_to_process_dict.values()

dict_values([[['20:20:26', 1.9947222222222223, 'fork:', 'parent', 'pid=3218', '->', 'clone', 'pid=5037', '(/usr/lib/tracker/tracker-miner-fs)'], ['20:20:30', 2.2266666666666666, 'fork:', 'parent', 'pid=2886', '->', 'clone', 'pid=5038', '(gpg-agent)'], ['20:20:30', 0.6494444444444444, 'fork:', 'parent', 'pid=2886', '->', 'clone', 'pid=5039', '(gpg-agent)'], ['20:20:30', 2.1106944444444444, 'exit:', 'pid=5038', 'exit_code=0', '(gpg-agent)'], ['20:20:30', 1.368472222222222, 'exit:', 'pid=5039', 'exit_code=0', '(gpg-agent)']], [['20:20:42', 5.051570247933884, 'fork:', 'parent', 'pid=4538', '->', 'clone', 'pid=5041', '(java)'], ['20:20:42', 3.2084297520661154, 'exit:', 'pid=5037', 'exit_code=0', '(/usr/lib/tracker/tracker-miner-fs)']], [['20:20:50', 8.31, 'exit:', 'pid=5040', 'exit_code=0', '(/usr/bin/whoopsie)']], [['20:21:02', 8.2, 'fork:', 'parent', 'pid=2014', '->', 'clone', 'pid=5042', '(/usr/bin/whoopsie)']], [], [['20:21:22', 8.02, 'fork:', 'parent', 'pid=3003', '->', 'clone', 'pid=5

In [8]:
consumption_pid_list = []

for List in timeStamp_to_process_dict.values():

  for  item in List:

    #modify item - append the process details together
    n = len(item)

    if (item[n-1][0] != '('):
      i = n-1

      #mash the whole process detail string together
      while (item[i][0] != '('):
        item[i-1] += " "
        item[i-1] += item[i]
        item.pop()
        i-=1

    #for pids
    if (len(item) == 9):
      #in this case, retain only the clone pid
      del item[3:7]
    
    else:
      #in this case, retain the pid but not the exit code
      del item[4]

    #finally, in the pid column, remove the "pid=" part from the string
    item[3] = int(item[3][4:])

    #append it to list
    consumption_pid_list.append(item)

consumption_pid_list

[['20:20:26',
  2.7427007299270074,
  'fork:',
  5037,
  '(/usr/lib/tracker/tracker-miner-fs)'],
 ['20:20:30', 1.8284671532846715, 'fork:', 5038, '(gpg-agent)'],
 ['20:20:30', 1.9198905109489048, 'fork:', 5039, '(gpg-agent)'],
 ['20:20:30', 1.676094890510949, 'exit:', 5038, '(gpg-agent)'],
 ['20:20:30', 0.18284671532846714, 'exit:', 5039, '(gpg-agent)'],
 ['20:20:42', 2.8320000000000003, 'fork:', 5041, '(java)'],
 ['20:20:42', 5.428, 'exit:', 5037, '(/usr/lib/tracker/tracker-miner-fs)'],
 ['20:20:50', 8.31, 'exit:', 5040, '(/usr/bin/whoopsie)'],
 ['20:21:02', 8.2, 'fork:', 5042, '(/usr/bin/whoopsie)'],
 ['20:21:22', 8.02, 'fork:', 5043, '(/usr/bin/gnome-shell)'],
 ['20:21:30', 0.8415272727272728, 'fork:', 5045, '(gpg-agent)'],
 ['20:21:30', 1.5379636363636364, 'exit:', 5044, '(gpg-agent)'],
 ['20:21:30', 1.944218181818182, 'exit:', 5045, '(gpg-agent)'],
 ['20:21:33', 1.8861818181818184, 'exit:', 4393, '(kworker/u16:2)'],
 ['20:21:33', 1.770109090909091, 'exit:', 332, '(kworker/7:2)'],


In [9]:
len(consumption_pid_list)

46

In [10]:
consumption_pid_df = pd.DataFrame(consumption_pid_list, columns = ["Timestamp", "Watts", "Process", "PID", "Message"])
consumption_pid_df

,Timestamp,Watts,Process,PID,Message
0,20:20:26,2.742701,fork:,5037,(/usr/lib/tracker/tracker-miner-fs)
1,20:20:30,1.828467,fork:,5038,(gpg-agent)
2,20:20:30,1.919891,fork:,5039,(gpg-agent)
3,20:20:30,1.676095,exit:,5038,(gpg-agent)
4,20:20:30,0.182847,exit:,5039,(gpg-agent)
5,20:20:42,2.832000,fork:,5041,(java)
6,20:20:42,5.428000,exit:,5037,(/usr/lib/tracker/tracker-miner-fs)
7,20:20:50,8.310000,exit:,5040,(/usr/bin/whoopsie)
8,20:21:02,8.200000,fork:,5042,(/usr/bin/whoopsie)
9,20:21:22,8.020000,fork:,5043,(/usr/bin/gnome-shell)


Change the path to whatever you want it to be in your google drive - this will output the **final measurements** including process ids and process details messages

In [11]:
pid_timestamp_measurements_path = '/content/drive/MyDrive/HSBC/march-to-sustainability/pid_timestamp_measurements.csv'

with open(pid_timestamp_measurements_path, 'w', encoding = 'utf-8-sig') as f1:
  consumption_pid_df.to_csv(f1)

#DON'T RERUN THIS BLOCK AFTER FIRST TIME

In [ ]:
print(power_stats)

[['Average', '0.2', '0.0', '0.1', '98.8', '0.9', '1.0', '352.3', '123.4', '2.1', '0.1', '2.5', '7.85'], ['StdDev', '0.1', '0.1', '0.1', '0.6', '0.6', '0.0', '86.1', '26.3', '1.6', '0.7', '2.1', '1.12'], ['Minimum', '0.1', '0.0', '0.1', '97.1', '0.2', '1.0', '261.2', '84.1', '0.0', '0.0', '0.0', '5.92'], ['Maximum', '0.5', '0.3', '0.4', '99.6', '2.5', '1.0', '722.4', '232.2', '6.0', '4.0', '9.0', '8.98']]
